In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm
import pandas as pd

%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

In [2]:
%matplotlib inline
import IPython.display
from ipywidgets import interact, interactive, fixed

# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage

import glob
import os
from tqdm import tqdm
# plt, plot, tqdm
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Packages we're using
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy.io import wavfile
from scipy.signal import butter, lfilter
import scipy.ndimage
from tqdm import tqdm

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [6]:
import numpy as np
import math
from PIL import Image
import time

In [7]:
path_train = './train'
path_test = './test'
path_ytrain = './y_train'
path_ytest = './y_test'

path_mat_train = './mat_train'
path_mat_test = './mat_test'
path_mat_ytrain = './mat_ytrain'
path_mat_ytest = './mat_ytest'

In [8]:
ROW = 257
COL = 62

In [9]:
INPUT_DIM = (ROW,COL,3)
INPUT_DIM[:2]

(257, 62)

In [10]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import gzip
%matplotlib inline
from keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D,concatenate
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Model
from keras.optimizers import RMSprop


In [11]:
FFT_LENGTH = 512
WINDOW_LENGTH = 512
WINDOW_STEP = int(WINDOW_LENGTH / 2)

phaseMax = 3.141592653589793 
phaseMin = -3.141592653589793
magnitudeMax = 2211683.973249525
magnitudeMin = 0.0

In [12]:
def amplifyMagnitudeByLog(d):
    return 188.301 * math.log10(d + 1)

def weakenAmplifiedMagnitude(d):
    return math.pow(10, d/188.301)-1

def generateLinearScale(magnitudePixels, phasePixels, 
                        magnitudeMin, magnitudeMax, phaseMin, phaseMax):
    height = magnitudePixels.shape[0]
    width = magnitudePixels.shape[1]
    magnitudeRange = magnitudeMax - magnitudeMin
    phaseRange = phaseMax - phaseMin
    rgbArray = np.zeros((height, width, 3), 'uint8')
    
    for w in range(width):
        for h in range(height):
            magnitudePixels[h,w] = (magnitudePixels[h,w] - magnitudeMin) / (magnitudeRange) * 255 * 2
            magnitudePixels[h,w] = amplifyMagnitudeByLog(magnitudePixels[h,w])
            phasePixels[h,w] = (phasePixels[h,w] - phaseMin) / (phaseRange) * 255
            red = 255 if magnitudePixels[h,w] > 255 else magnitudePixels[h,w]
            green = (magnitudePixels[h,w] - 255) if magnitudePixels[h,w] > 255 else 0
            blue = phasePixels[h,w]
            rgbArray[h,w,0] = int(red)
            rgbArray[h,w,1] = int(green)
            rgbArray[h,w,2] = int(blue)
    return rgbArray

def recoverLinearScale(rgbArray, magnitudeMin, magnitudeMax, 
                       phaseMin, phaseMax):
    width = rgbArray.shape[1]
    height = rgbArray.shape[0]
#     print(phaseMax,phaseMin)
    magnitudeVals = rgbArray[:,:,0].astype(float) + rgbArray[:,:,1].astype(float)
    phaseVals = rgbArray[:,:,2].astype(float)
    phaseRange = phaseMax - phaseMin
    magnitudeRange = magnitudeMax - magnitudeMin
    
#     print(magnitudeVals)
#     print(phaseVals)
#     print(phaseRange)
#     print(magnitudeRange)
    
    for w in range(width):
        for h in range(height):
            phaseVals[h,w] = (phaseVals[h,w] / 255 * phaseRange) + phaseMin
            magnitudeVals[h,w] = weakenAmplifiedMagnitude(magnitudeVals[h,w])
            magnitudeVals[h,w] = (magnitudeVals[h,w] / (255*2) * magnitudeRange) + magnitudeMin
    return magnitudeVals, phaseVals

In [13]:
def recoverSignalFromSpectrogram(numpyarray):
#     img = Image.open(filePath)
    data = np.array( numpyarray, dtype='uint8' )
    width = data.shape[1]
    height = data.shape[0]
    
#     print(phaseMax,phaseMin)

    magnitudeVals, phaseVals \
    = recoverLinearScale(data, magnitudeMin, magnitudeMax, phaseMin, phaseMax)
    
#     print(magnitudeVals,phaseVals)
    
    recovered = np.zeros(WINDOW_LENGTH * width // 2 + WINDOW_STEP, dtype=np.int16)
    recovered = np.array(recovered,dtype=np.int16)
    
    for w in range(width):
        toInverse = np.zeros(height, dtype=np.complex_)
        for h in range(height):
            magnitude = magnitudeVals[height-h-1,w]
            phase = phaseVals[height-h-1,w]
            toInverse[h] = magnitude * math.cos(phase) + (1j * magnitude * math.sin(phase))
        signal = np.fft.irfft(toInverse)
        recovered[w*WINDOW_STEP:w*WINDOW_STEP + WINDOW_LENGTH] += signal[:WINDOW_LENGTH].astype(np.int16)
    return recovered

In [14]:
nb_train_samples = 7000 # change here
nb_test_samples = 105

train_batch_size = [5,4,3,2]
test_batch_size = [5,4,3,2]

epochs = 30
inChannel = 3
x, y = INPUT_DIM[:2]
input_img = Input(shape = (x, y, inChannel))

In [15]:
train_image = []
path = path_mat_train
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    train_image.append(os.path.join(filename))
    
ytrain_image = []
path = path_mat_ytrain
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    ytrain_image.append(os.path.join(filename))
    
test_image = []
path = path_mat_test
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    test_image.append(os.path.join(filename))
    
ytest_image = []
path = path_mat_ytest
for filename in tqdm(glob.glob(os.path.join(path, '*.npy'))):
    ytest_image.append(os.path.join(filename))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105/105 [00:00<00:00, 105283.75it/s]


In [16]:
print(train_image[0],ytrain_image[0])

./mat_train\103-1240-0005.npy ./mat_ytrain\103-1240-0005.npy


In [17]:
print(test_image[0],ytest_image[0])

./mat_test\1034-121119-0049.npy ./mat_ytest\1034-121119-0049.npy


In [18]:
def data_gen_train(train_batch_size):
    while True:
        for start in range(0,nb_train_samples,train_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + train_batch_size, nb_train_samples)
            for img_path in range(start, end):
                img_train = np.load(train_image[img_path])
                img_train = img_train/255
                x_batch.append(img_train)
                img_ytrain = np.load(ytrain_image[img_path])
                img_ytrain = img_ytrain/255
                y_batch.append(img_ytrain)
            yield (np.array(x_batch), np.array(y_batch)) 

In [19]:
def data_gen_test(test_batch_size):
    while True:
        for start in range(0,nb_test_samples,test_batch_size):
            x_batch = []
            y_batch = []
            end = min(start + test_batch_size, nb_test_samples)
            for img_path in range(start, end):
                img_test = np.load(test_image[img_path])
                img_test = img_test/255
                x_batch.append(img_test)
                img_ytest= np.load(ytest_image[img_path])
                img_ytest = img_ytest/255
                y_batch.append(img_ytest)
            yield (np.array(x_batch), np.array(y_batch)) 

# Model Architecture 1

In [20]:
# Not using this architecture
# Attempt to reduce network size, latency

In [21]:
def autoencoder(input_img):
    #encoder
    conv1 = Conv2D(32, (2, 2), strides=(1, 1), activation='relu', padding='same')(input_img)
    pool1 = MaxPooling2D(pool_size=(1, 1))(conv1)
    pool1 = Dropout(0.5)(pool1)
    conv2 = Conv2D(64, (2, 2), strides=(1, 1), activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(1, 1))(conv2)
    pool2 = Dropout(0.5)(pool2)
    conv3 = Conv2D(128, (2, 2), strides=(1, 1), activation='relu', padding='same')(pool2)
    pool3 = MaxPooling2D(pool_size=(1, 1))(conv3)
    pool3 = Dropout(0.5)(pool3)
    conv4 = Conv2D(256, (2, 2), strides=(1, 1), activation='relu', padding='same')(pool3)
    
    #decoder
    conv5 = Conv2D(256, (2, 2),strides=(1, 1), activation='relu', padding='same')(conv4)
    skip1 = concatenate([conv5, conv4], name='skip1')
    up1 = UpSampling2D((1,1))(skip1)
    up1 = Dropout(0.5)(up1)
    conv6 = Conv2D(128, (2, 2),strides=(1, 1), activation='relu', padding='same')(conv5)
    skip2 = concatenate([conv6, conv3], name='skip2')
    up2 = UpSampling2D((1,1))(skip2)
    up2 = Dropout(0.5)(up2)
    conv7 = Conv2D(64, (2, 2),strides=(1, 1), activation='relu', padding='same')(up2)
    skip3 = concatenate([conv7, conv2], name='skip3')
    up3 = UpSampling2D((1,1))(skip3)
    up3 = Dropout(0.5)(up3)
    decoded = Conv2D(3, (2, 2),strides=(1, 1), activation='sigmoid', padding='same')(up3)
    return decoded

In [22]:
from keras.optimizers import RMSprop, Adam
opt = RMSprop( learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,name='RMSprop')
autoencoder = Model(input_img, autoencoder(input_img))
autoencoder.compile(loss='mse', optimizer = opt, metrics=['accuracy'])
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 257, 62, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 257, 62, 32)  416         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 257, 62, 32)  0           conv2d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 257, 62, 32)  0           max_pooling2d[0][0]              
______________________________________________________________________________________________

In [23]:
# from keras.optimizers import SGD
# opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# autoencoder = Model(input_img, autoencoder(input_img))
# autoencoder.compile(loss='categorical_crossentropy', optimizer = opt, metrics=['accuracy'])

# Model Architecture 2

In [24]:
import keras.layers as layers
import keras.models as models
from keras.initializers import orthogonal


def Conv2DLayer(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2D(filters, kernel_size=kernel, strides=strides, padding=padding,
                      kernel_initializer=kernel_init, name=prefix+'conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x

def Transpose_Conv2D(x, filters, kernel, strides, padding, block_id, kernel_init=orthogonal()):
    prefix = f'block_{block_id}_'
    x = layers.Conv2DTranspose(filters, kernel_size=kernel, strides=strides, padding=padding,
                               kernel_initializer=kernel_init, name=prefix+'de-conv')(x)
    x = layers.LeakyReLU(name=prefix+'lrelu')(x)
    x = layers.Dropout(0.2, name=prefix+'drop')((x))
    x = layers.BatchNormalization(name=prefix+'conv_bn')(x)
    return x



def AutoEncdoer(input_shape):
    inputs = layers.Input(shape=input_shape)
    
    # 256 x 256
    conv1 = Conv2DLayer(inputs, 64, 3, strides=1, padding='same', block_id=1)
    conv2 = Conv2DLayer(conv1, 64, 3, strides=1, padding='same', block_id=2)
    
    # 128 x 128
    conv3 = Conv2DLayer(conv2, 128, 5, strides=1, padding='same', block_id=3)
    
    # 64 x 64
    conv4 = Conv2DLayer(conv3, 128, 3, strides=1, padding='same', block_id=4)
    conv5 = Conv2DLayer(conv4, 256, 5, strides=1, padding='same', block_id=5)
    
    # 32 x 32
    conv6 = Conv2DLayer(conv5, 512, 3, strides=1, padding='same', block_id=6)
    
    # 16 x 16
    deconv1 = Transpose_Conv2D(conv6, 512, 3, strides=1, padding='same', block_id=7)
    
    # 32 x 32
    skip1 = layers.concatenate([deconv1, conv5], name='skip1')
    conv7 = Conv2DLayer(skip1, 256, 3, strides=1, padding='same', block_id=8)
    deconv2 = Transpose_Conv2D(conv7, 128, 3, strides=1, padding='same', block_id=9)
    
    # 64 x 64
    skip2 = layers.concatenate([deconv2, conv3], name='skip2')
    conv8 = Conv2DLayer(skip2, 128, 5, strides=1, padding='same', block_id=10)
    deconv3 = Transpose_Conv2D(conv8, 64, 3, strides=1, padding='same', block_id=11)
    
    # 128 x 128
    skip3 = layers.concatenate([deconv3, conv2], name='skip3')
    conv9 = Conv2DLayer(skip3, 64, 5, strides=1, padding='same', block_id=12)
    deconv4 = Transpose_Conv2D(conv9, 64, 3, strides=1, padding='same', block_id=13)
    
    # 256 x 256
    skip3 = layers.concatenate([deconv4, conv1])
    conv10 = layers.Conv2D(3, 3, strides=1, padding='same', activation='sigmoid',
                       kernel_initializer=orthogonal(), name='final_conv')(skip3)

    
    return models.Model(inputs=inputs, outputs=conv10)

In [25]:
from keras.optimizers import SGD, Adam, RMSprop 
opt = RMSprop( learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,name='RMSprop')
# opt = Adam(lr=0.001)
# opt = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
autoencoder = AutoEncdoer((ROW,COL, 3))
autoencoder.compile(optimizer=opt, loss=['mae','mse'], metrics=['mse','accuracy'])

In [26]:
autoencoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 257, 62, 3)] 0                                            
__________________________________________________________________________________________________
block_1_conv (Conv2D)           (None, 257, 62, 64)  1792        input_2[0][0]                    
__________________________________________________________________________________________________
block_1_lrelu (LeakyReLU)       (None, 257, 62, 64)  0           block_1_conv[0][0]               
__________________________________________________________________________________________________
block_1_drop (Dropout)          (None, 257, 62, 64)  0           block_1_lrelu[0][0]              
____________________________________________________________________________________________

In [27]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [28]:
import os
import random

import tensorflow as tf
from tensorflow import keras


from tensorflow.python.keras.saving import saving_utils as _saving_utils
from tensorflow.python.framework import convert_to_constants as convert_variables_to_constants_v2

print(tf.version.VERSION)

2.2.0


In [29]:
PERIOD = 5

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "modelcheckpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 10 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=PERIOD)

In [30]:
# Save the weights using the `checkpoint_path` format
autoencoder.save_weights(checkpoint_path.format(epoch=0))

In [31]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'modelcheckpoints\\cp-0000.ckpt'

In [32]:
checkpoint_dir

'modelcheckpoints'

In [33]:
# Load the previously saved weights
autoencoder.load_weights(latest)

In [34]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, autoencoder_train
                     ,epoch, logs=None):
        rate = 16000
        for j in range(5):
            r_num = random.randint(0, nb_test_samples-1)
            test_file = test_image[r_num]
            img_test = np.load(test_file)
            img_test = img_test/255
            img_test = img_test.reshape(-1, ROW,COL,3)
            decoded_imgs = autoencoder.predict(img_test) #predict
            decoded_imgs = decoded_imgs.reshape(ROW,COL,3)
            decoded_imgs = decoded_imgs*255
            decoded_imgs = decoded_imgs.astype(np.int16)
            data = recoverSignalFromSpectrogram(decoded_imgs)
            scipy.io.wavfile.write('./'+"predict_{}".format(j)+'.wav', rate, data)
        autoencoder.save('model.h5')

In [35]:
from matplotlib import pyplot

In [ ]:
for i in range(len(train_batch_size)):
    autoencoder_train = autoencoder.fit(data_gen_train(train_batch_size[i]),
                                    epochs= epochs,
                                    verbose=1,
                                    steps_per_epoch= nb_train_samples // train_batch_size[i],
                                    validation_data= data_gen_test(test_batch_size[i]),
                                    validation_steps=nb_test_samples // test_batch_size[i],
                                    callbacks=[cp_callback,CustomCallback()])
    pyplot.subplot(211)
    pyplot.title('Loss')
    pyplot.plot(autoencoder_train.history['loss'], label='train')
    pyplot.plot(autoencoder_train.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()
    pyplot.subplot(212)
    pyplot.title('Mean Squared Error')
    pyplot.plot(autoencoder_train.history['mean_squared_error'], label='train')
    pyplot.plot(autoencoder_train.history['val_mean_squared_error'], label='test')
    pyplot.legend()
    pyplot.show()
    # list all data in history
    print(autoencoder_train.history.keys())
    # summarize history for accuracy
    plt.plot(autoencoder_train.history['accuracy'])
    plt.plot(autoencoder_train.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
#     # summarize history for loss
#     plt.plot(autoencoder_train.history['loss'])
#     plt.plot(autoencoder_train.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()

# evaluate the model=
#     _, train_acc = autoencoder.evaluate(data_gen_train(train_batch_size[i]), verbose=0)
#     _, test_acc = autoencoder.evaluate(data_gen_test(test_batch_size[i]), verbose=0)
#     print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot training history


Epoch 1/30
1400/1400 [==============================] - 13646s 10s/step - loss: 0.0886 - mse: 0.0338 - accuracy: 0.9122 - val_loss: 0.0965 - val_mse: 0.0335 - val_accuracy: 0.8752
Epoch 2/30
1400/1400 [==============================] - 13801s 10s/step - loss: 0.0781 - mse: 0.0270 - accuracy: 0.9265 - val_loss: 0.0957 - val_mse: 0.0326 - val_accuracy: 0.8777
Epoch 3/30
1311/1400 [===========================>..] - ETA: 33:34 - loss: 0.0770 - mse: 0.0265 - accuracy: 0.9277